# Работа с Excel

In [1]:
!pip install xlwings
!pip install xlsxwriter

In [2]:
import xlwings as xw
import pandas as pd
from xlwings.constants import AutoFillType
import numpy as np
from xlsxwriter import Workbook

In [3]:
xw.__version__

'0.29.1'

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [4]:
rw=pd.read_csv('reviews_sample.csv', delimiter=',', index_col=0)
rp=pd.read_csv('recipes_sample.csv', delimiter=',')
del rp['contributor_id']
del rp['n_steps']
rp

,name,id,minutes,submitted,description,n_ingredients
0,george s at the cove black bean soup,44123,90,2002-10-25,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,2003-07-26,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,2002-08-29,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,2002-07-27,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,2004-02-23,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,2007-11-25,this is based on a french recipe but i changed...,10.0
29996,zwetschgenkuchen bavarian plum cake,386977,240,2009-08-24,"this is a traditional fresh plum cake, thought...",11.0
29997,zwiebelkuchen southwest german onion cake,103312,75,2004-11-03,this is a traditional late summer early fall s...,NaN
29998,zydeco soup,486161,60,2012-08-29,this is a delicious soup that i originally fou...,NaN


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [5]:
s2=rp.sample(frac=.05)
s1=rw.sample(frac=.05)
with pd.ExcelWriter(r'C:\Users\1\Downloads\04_tables\data\recipes2.xlsx', engine='xlsxwriter') as writer:
    s2.to_excel(writer, sheet_name='рецепты')
    s1.to_excel(writer, sheet_name='отзывы')

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [6]:
wb = xw.Book(r'C:\Users\1\Downloads\04_tables\data\recipes2.xlsx')
sht = wb.sheets['рецепты']
sht.range('H1').value = 'seconds_assign'
sht.range('H2').options(transpose = True).value = [k * 60 for k in s2['minutes']]

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [8]:
sht['I2:I1501'].formula='=D2*60'
sht['I1'].value='seconds_formula'

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [9]:
sht.range('H1:I1').api.Font.Bold = True
sht.range('H1:I1').api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [10]:
for zv in sht['D2:D1501']:
    if zv.value<5:
        zv.color=(0,128,0)
    elif zv.value<=10:
        zv.color=(255,204,0)
    else:
        zv.color=(128, 0, 0)

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [14]:
sht.range('J1').value = 'n_reviews'
fml = sht.range('J2').formula = '=COUNTIF(отзывы!C:C,C2)'
sht.range('J2:J1501').formula = fml

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [ ]:
def validate():
    wb = xw.Book(r'C:\Users\1\Downloads\04_tables\data\recipes2.xlsx')
    sht = wb.sheets['отзывы']
    for i in range(2, 6336):
        if sht.range(f'E{i}').value < 0 or sht.range(f'E{i}').value > 5 or sht.range(f'C{i}').value not in wb.sheets['рецепты'].range('A2:A1501').value:
            sht.range(f'A{i}:Z{i}').color = (128, 0, 0)
validate()

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

In [46]:
import csv
with open('recipes_model.csv','r') as f:
    rec_mod = csv.reader(f, delimiter = '\t')
    rec_data = list(rec_mod)
model = xw.Book()
model_sheet = model.sheets.add('Модель')
model_sheet.range('A2').value = rec_data
model.save('recipes_model1.xlsx')
model.close()